# Target Definition

Using the date and reported EPS at a given date to create the target which consists in the EPS x years later.

## Import Libraries

In [1]:
import pandas as pd

## Import Data

In [2]:
data = pd.read_csv('../data/preprocessed/preprocessed_data.csv')
data.head()

,fiscalDateEnding,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,...,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,reportedEPS,Sector,Industry
0,2024-11-02,48228277000,5.484654e+09,1.991342e+09,2.363164e+09,1.447687e+09,1.336331e+09,4.274362e+10,3.415550e+09,3.772438e+09,...,142067000,322227000.0,1.635273e+09,1.638319e+09,2032798000,3.732798e+09,1635273000,6.38,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES
1,2023-10-28,48794478000,4.384022e+09,9.580610e+08,9.580610e+08,1.642214e+09,1.469734e+09,4.441046e+10,3.219157e+09,3.424775e+09,...,293424000,264641000.0,3.314579e+09,3.324429e+09,3872644000,5.872644e+09,3314579000,10.08,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES
2,2022-10-29,50302350000,4.937992e+09,1.470572e+09,1.470572e+09,1.399914e+09,1.800462e+09,4.536436e+10,2.401304e+09,3.148203e+09,...,350188000,200408000.0,2.748561e+09,2.736974e+09,3299157000,5.313357e+09,2748561000,9.59,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES
3,2021-10-30,52322071000,5.378317e+09,1.977964e+09,1.977964e+09,1.200610e+09,1.459056e+09,4.694375e+10,1.979051e+09,2.956246e+09,...,-61708000,399975000.0,1.390422e+09,1.453318e+09,1513539000,2.356898e+09,1390422000,6.43,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES
4,2020-10-31,21468603000,2.517688e+09,1.055860e+09,1.055860e+09,6.082600e+08,7.375360e+08,1.889568e+10,1.120561e+09,2.765095e+09,...,90856000,193305000.0,1.220761e+09,1.161478e+09,1504922000,2.082070e+09,1220761000,4.91,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES


## Extract the year information

We need to extract the year from the date so that we then will be able to retrieve the EPS x years later (which is what we want as a target).

In [3]:
# we need to extract the year from the date
data['year'] = data.fiscalDateEnding.str[:4].astype(int)
data[['fiscalDateEnding', 'year']].head()


,fiscalDateEnding,year
0,2024-11-02,2024
1,2023-10-28,2023
2,2022-10-29,2022
3,2021-10-30,2021
4,2020-10-31,2020


There is one thing we need to pay attention to: we do not want to have twice the same year for a given symbol. Let's take a look.

In [4]:
# first, let's create a temporary dataset with only what we need to do our verifications
temp = data.copy()
temp = temp[['symbol', 'fiscalDateEnding', 'year']]
temp.head()

,symbol,fiscalDateEnding,year
0,ADI,2024-11-02,2024
1,ADI,2023-10-28,2023
2,ADI,2022-10-29,2022
3,ADI,2021-10-30,2021
4,ADI,2020-10-31,2020


In [5]:
# then, we need the list of symbols in the data
symbols = temp['symbol'].unique()
len(symbols)

101

In [6]:
# for each symbol, we will look at the list of years we have
# then we can compare the length of the list of years [2024, 2023,...,2016]
# with the length of the set of this same list
# this works because a mathematical set only contains unique values.

# exemple
mylist = ['a', 'a', 'a', 'b', 'c']
myset = set(mylist)
print(f"my list: {mylist} | my set: {myset}")

my list: ['a', 'a', 'a', 'b', 'c'] | my set: {'b', 'a', 'c'}


In [7]:
# in our case

for sym in symbols:
    listofyears = temp[temp['symbol'] == sym]['year'].values
    setofyears = set(listofyears)
    if len(listofyears) != len(setofyears):
        print(f"symbol: {sym} | list length: {len(listofyears)} | set length: {len(setofyears)} | duplicate(s): {len(listofyears) - len(setofyears)}")

symbol: JNJ | list length: 17 | set length: 14 | duplicate(s): 3


We do, it seems, have one symbol with more than one time the same year in the dataset. Let's investigate to see if we can come up with a solution.

In [8]:
temp[temp['symbol'] == 'JNJ']

,symbol,fiscalDateEnding,year
1283,JNJ,2023-12-31,2023
1284,JNJ,2022-12-31,2022
1285,JNJ,2021-12-31,2021
1286,JNJ,2020-12-31,2020
1287,JNJ,2019-12-31,2019
1288,JNJ,2018-12-31,2018
1289,JNJ,2017-12-31,2017
1290,JNJ,2016-12-31,2016
1291,JNJ,2015-12-31,2015
1292,JNJ,2014-12-31,2014


In this particular case, the issue comes from the data scources themselves. Indeed, there are two entries for 2012 in both the balance sheet data and the income statement data. Here, we will keep the largest entry.

In [9]:
data = data.groupby(['year', 'symbol'], as_index=False).max()

In [10]:
data[data['symbol'] == 'JNJ']

,year,symbol,fiscalDateEnding,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,...,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,reportedEPS,Sector,Industry
108,2010,JNJ,2010-12-31,94682000000,3.954100e+10,1.581000e+10,1.942500e+10,5.180000e+09,9.646000e+09,4.515400e+10,...,3489000000,4.510000e+08,1.226600e+10,1.339900e+10,16206000000,1.652700e+10,12266000000,4.76,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS
192,2011,JNJ,2011-12-31,102908000000,4.730700e+10,1.935500e+10,2.765800e+10,5.378000e+09,9.774000e+09,4.584800e+10,...,3613000000,4.550000e+08,1.333400e+10,1.286100e+10,17402000000,1.815000e+10,13334000000,5.00,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS
281,2012,JNJ,2012-12-31,121347000000,5.431600e+10,2.454200e+10,3.226100e+10,7.495000e+09,1.130900e+10,7.069000e+10,...,3261000000,5.710000e+08,1.051400e+10,1.067500e+10,14646000000,1.579200e+10,10853000000,5.11,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS
373,2013,JNJ,2013-12-31,132683000000,5.640700e+10,2.092700e+10,2.920600e+10,7.878000e+09,1.171300e+10,7.240400e+10,...,1640000000,4.820000e+08,1.383100e+10,1.678100e+10,15953000000,1.731600e+10,13831000000,5.52,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS
467,2014,JNJ,2014-12-31,130358000000,5.574400e+10,1.452300e+10,3.308900e+10,8.184000e+09,1.098500e+10,6.852900e+10,...,4240000000,5.330000e+08,1.632300e+10,8.461000e+09,21096000000,2.249400e+10,16323000000,5.97,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS
562,2015,JNJ,2015-12-31,133411000000,6.021000e+10,1.373200e+10,3.837600e+10,8.053000e+09,1.073400e+10,6.771100e+10,...,3787000000,5.520000e+08,1.540900e+10,1.296600e+10,19748000000,2.094800e+10,15409000000,6.20,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS
657,2016,JNJ,2016-12-31,141208000000,6.503200e+10,1.897200e+10,4.190700e+10,8.144000e+09,1.169900e+10,7.002800e+10,...,3263000000,7.260000e+08,1.654000e+10,1.480400e+10,20529000000,2.172900e+10,16540000000,6.68,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS
752,2017,JNJ,2017-12-31,157303000000,4.308800e+10,1.782400e+10,1.829600e+10,8.765000e+09,1.349000e+10,1.076360e+11,...,16373000000,9.340000e+08,1.300000e+09,3.002000e+09,18607000000,2.160700e+10,1300000000,7.30,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS
847,2018,JNJ,2018-12-31,152954000000,4.603300e+10,1.810700e+10,1.968700e+10,8.599000e+09,1.409800e+10,9.975600e+10,...,2702000000,1.005000e+09,1.529700e+10,1.350600e+10,19004000000,2.340400e+10,15297000000,8.18,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS
944,2019,JNJ,2019-12-31,157728000000,4.527400e+10,1.730500e+10,2.043500e+10,9.020000e+09,1.448100e+10,1.051860e+11,...,2209000000,3.180000e+08,1.511900e+10,1.445000e+10,17646000000,2.214600e+10,15119000000,8.68,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS


## Building the Target Variable

We can create a copy of the dataset and shift the year information by a constant than merge the original dataset and its altered copy to get ourselves a target variable.

In [ ]:
# Let's say we want to predict the EPS 5 years from the observed data year
PREDICTION_HORIZON = 5

target_data = data.copy()
target_data = target_data[['symbol', 'year', 'reportedEPS']]

# we can substract the prediction horizon from the year
target_data['year'] = target_data['year'] - PREDICTION_HORIZON

# let's also rename the columns
target_data.rename(columns={'reportedEPS' : 'futureEPS'}, inplace=True)

# Now we can merge the original dataset with this new one
data = data.merge(target_data, on=['symbol', 'year'], how='left')
data.head()

,year,symbol,fiscalDateEnding,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,...,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,reportedEPS,Sector,Industry,futureEPS
0,2004,BLK,2004-12-31,1145235000,8.674240e+08,4.576730e+08,4.576730e+08,15549000.0,NaN,2.778110e+08,...,NaN,NaN,NaN,142517000,1.632030e+08,143141000,2.73,FINANCE,"SECURITY BROKERS, DEALERS & FLOTATION COMPANIES",7.05
1,2005,BLK,2005-12-31,1848000000,1.234567e+09,4.842230e+08,4.842230e+08,61882000.0,NaN,6.134330e+08,...,NaN,NaN,NaN,297403000,3.283050e+08,233908000,4.04,FINANCE,"SECURITY BROKERS, DEALERS & FLOTATION COMPANIES",10.94
2,2006,BLK,2006-12-31,20469492000,2.124670e+09,1.160304e+09,1.160304e+09,472483000.0,NaN,1.135423e+10,...,NaN,NaN,NaN,405451000,4.782600e+08,322602000,5.09,FINANCE,"SECURITY BROKERS, DEALERS & FLOTATION COMPANIES",11.85
3,2007,BLK,2007-12-31,22561515000,3.241842e+09,1.656200e+09,1.656200e+09,395006000.0,NaN,1.931967e+10,...,NaN,NaN,NaN,1368130000,1.566954e+09,995272000,8.20,FINANCE,"SECURITY BROKERS, DEALERS & FLOTATION COMPANIES",13.69
4,2008,BLK,2008-12-31,19924000000,3.242000e+09,2.032000e+09,2.032000e+09,122000000.0,NaN,1.668200e+10,...,NaN,NaN,NaN,1695671000,1.971420e+09,786419000,6.43,FINANCE,"SECURITY BROKERS, DEALERS & FLOTATION COMPANIES",16.60


At the time of writing this (2024), we do not know what would be the EPS for a 2023 or 2024 observations five years from now (i.e. in 2028 and 2029). We will therefore get some NaN values which we preemptively deal with by using an inner joint merge.

## Saving the Dataset

In [12]:
data.to_csv('../data/preprocessed/preprocessed_data_with_target.csv', index=False)